<a href="https://colab.research.google.com/github/lonsst/ML_practice/blob/main/LSTM_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install requests

In [ ]:
pip install nltk

Задание 1.

Загрузите текст из произведений Ницше ('nietzsche.txt', origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt').
Выведете следующее:
А) длину всего корпуса;
Б) количество предложений;
В) сколько всего символов используется?

In [ ]:
import requests
import nltk
from nltk.tokenize import sent_tokenize

In [ ]:
from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.optimizers import RMSprop
#from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
path = './nietzsche.txt'
nietzsche_text = open(path).read().lower()

Длина корпуса

In [ ]:
corpus_length = len(nietzsche_text)
print(f'Длина корпуса: {corpus_length} символов')

Длина корпуса: 600893 символов


Количество предложений

In [ ]:
sentences = sent_tokenize(nietzsche_text)
num_sentences = len(sentences)
print(f'Количество предложений: {num_sentences}')

Количество предложений: 2864


Сколько символов используется

In [ ]:
unique_characters = sorted(list(set(nietzsche_text)))
num_unique_characters = len(unique_characters)
print(f'Количество уникальных символов: {num_unique_characters}')

Количество уникальных символов: 57


In [ ]:
char_indices = dict((c, i) for i, c in enumerate(unique_characters))
indices_char = dict((i, c) for i, c in enumerate(unique_characters))

Задание 2.

Сократите текст наполовину избыточными последовательностями символов maxlen

In [ ]:
import numpy as np

In [ ]:
maxlen = 40
step = 3
sentences = []
next_chars = []

for i in range(0, len(nietzsche_text) - maxlen, step):
    sentences.append(nietzsche_text[i: i + maxlen])
    next_chars.append(nietzsche_text[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(unique_characters)), dtype=np.bool)
y = np.zeros((len(sentences), len(unique_characters)), dtype=np.bool)
for i, (sentence, next_char) in enumerate(zip(sentences, next_chars)):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_char]] = 1


nb sequences: 200285
Vectorization...


<ipython-input-11-05b7409846d0>:12: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  x = np.zeros((len(sentences), maxlen, len(unique_characters)), dtype=np.bool)
<ipython-input-11-05b7409846d0>:13: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y = np.zeros((len(sentences), len(unique_characters)), dtype=np.bool)


Задание 3.

Создайте модель LSTM для генерации текста

А) Напишите вспомогательную функцию для выборки индекса из массива вероятностей

Б) Напишите функцию, которая будет вызываться в конце каждой эпохи и печатать сгенерированный текст

В) Запустите модель на обучение




In [ ]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [ ]:
def generate_text(model, seed_text, length, temperature):
    generated_text = seed_text
    for _ in range(length):
        x_pred = np.zeros((1, maxlen, len(unique_characters)))
        for t, char in enumerate(seed_text):
            x_pred[0, t, char_indices[char]] = 1.0
        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, temperature)
        next_char = indices_char[next_index]
        generated_text += next_char
        seed_text = seed_text[1:] + next_char
    return generated_text

In [ ]:
def on_epoch_end(epoch, _):
    print()
    print(f'\n----- Эпоха {epoch + 1} завершена. Генерируем текст:')

    start_index = random.randint(0, len(nietzsche_text) - maxlen - 1)
    seed_text = nietzsche_text[start_index: start_index + maxlen]

    for temperature in [0.2, 0.5, 1.0]:
        print('----- Temperature:', temperature)
        generated_text = generate_text(model, seed_text, 400, temperature)
        print(generated_text)

In [ ]:
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(unique_characters))))
model.add(Dense(len(unique_characters), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

In [ ]:
model.fit(x, y, batch_size=128, epochs=10, callbacks=[LambdaCallback(on_epoch_end=on_epoch_end)])

Epoch 1/10
1564/1565 [============================>.] - ETA: 0s - loss: 2.5529

----- Эпоха 1 завершена. Генерируем текст:
----- Temperature: 0.2
a man could bear and take upon himself, the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the hererest of alle the the the the the the the the the mat of moment of the the the the the the the the the the the the the the the the the the the the the the the the the the the the besting of the sereste the the the the mand the the the the the the the sore the the the the p
----- Temperature: 0.5
a man could bear and take upon himself, the int and and raperente the the the and mate in them an amene wore fo ang of the the that the thes of whe pmat not at of and tor the mad the he the the cion of and as the of the in the cortist and mores and and in the core ous of in the phithian al the tho lara and ind the hall and is in the tore and af be the prof and of memese of the corlonath 

Пример скрипта для генерации текста из произведений Ницше.
Требуется не менее 20 эпох, прежде чем сгенерированный текст
начнет звучать связно.
Рекомендуется запускать этот скрипт на графическом процессоре, так как рекуррентные
сети требуют довольно больших вычислительных затрат.

In [ ]:
from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.optimizers import RMSprop
#from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io

In [ ]:
url = 'https://ru.wikibooks.org/wiki/%D0%A2%D0%B5%D1%85%D0%BD%D0%B8%D0%BA%D0%B0_%D0%B8_%D1%82%D0%B5%D1%85%D0%BD%D0%BE%D0%BB%D0%BE%D0%B3%D0%B8%D1%8F_%D1%81%D1%80%D0%B5%D0%B4%D1%81%D1%82%D0%B2_%D0%BC%D0%B0%D1%81%D1%81%D0%BE%D0%B2%D0%BE%D0%B9_%D0%B8%D0%BD%D1%84%D0%'
response = requests.get(url)
wiki_text = response.text

In [ ]:
url = 'https://www.kaggle.com/datasets/dhruvildave/wikibooks-dataset/wikibooks-dataset/russian-wikibooks/ru-books-dataset.csv'
response = requests.get(url)
wiki_text = response.text

In [ ]:
corpus_length = len(wiki_text)
corpus_length

5222

In [ ]:
sentences = sent_tokenize(wiki_text)
num_sentences = len(sentences)
num_sentences

3

In [ ]:
chars = sorted(list(set(wiki_text)))
num_chars = len(chars)
print(f'Количество уникальных символов: {num_chars}')

Количество уникальных символов: 91


In [ ]:
char_indices = dict((char, i) for i, char in enumerate(chars))
indices_char = dict((i, char) for i, char in enumerate(chars))

maxlen = 40
step = 3
sentences = []
next_chars = []

for i in range(0, len(wiki_text) - maxlen, step):
    sentences.append(wiki_text[i: i + maxlen])
    next_chars.append(wiki_text[i + maxlen])


x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)

for i, (sentence, next_char) in enumerate(zip(sentences, next_chars)):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_char]] = 1

<ipython-input-82-b30068a06c58>:14: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
<ipython-input-82-b30068a06c58>:15: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y = np.zeros((len(sentences), len(chars)), dtype=np.bool)


In [ ]:
def generate_text(model, seed_text, length, temperature):
    generated_text = seed_text
    for _ in range(length):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(seed_text):
            x_pred[0, t, char_indices[char]] = 1.0
        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, temperature)
        next_char = indices_char[next_index]
        generated_text += next_char
        seed_text = seed_text[1:] + next_char
    return generated_text

def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def on_epoch_end(epoch, _):
    print()
    print('----- Эпоха {} завершена. Генерируем текст:'.format(epoch + 1))

    start_index = random.randint(0, len(wiki_text) - maxlen - 1)
    seed_text = wiki_text[start_index: start_index + maxlen]

    for temperature in [0.2, 0.5, 1.0]:
        print('----- Temperature:', temperature)
        generated_text = generate_text(model, seed_text, 400, temperature)
        print(generated_text)

In [ ]:
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

In [ ]:
model.fit(x, y, batch_size=128, epochs=10, callbacks=[LambdaCallback(on_epoch_end=on_epoch_end)])

Epoch 1/10
13/14 [==========================>...] - ETA: 0s - loss: 4.4311
----- Эпоха 1 завершена. Генерируем текст:
----- Temperature: 0.2
"4oE4RdSyuAjtdxg7aItHLA==" src="/static/                                                                                                                                                                                                                                                                                                                                                                                                                
----- Temperature: 0.5
 e cK
   /       cp  s    s   ai  s" f        
f  (    s    a    J 
  li   "    C  E c     { a  rc      c  
 s       a       sp     s   e     / 
y       
----- Temperature: 1.0
"4oE4RdSyuAjtdxg7aItHLA==" src="/static/n"txc
dB(LE?Ws&" Zo)Asnv:a  n na6.Gp#ya';3i  _
M'r<#+spc
o ci e
4 = 
.5h6 .@
0iXf.x6  V  7
TpJ"
2pLhQkd
>Hs8psnlca<etAu{t
| e;J0M
a0)#"}z

/A a?v),n tU 
 E"Wa
  cd_
 s
14/14 [===

KeyboardInterrupt: ignored